In [1]:
import sys, os, git, time

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/python-codes/")

from InitializeSimulation import InitializeSimulation
from Utilities import Utilities
from Outputs import Outputs
from MolecularDynamics import MolecularDynamics
from MonteCarlo import MonteCarlo

import numpy as np

In [2]:
self = MolecularDynamics(number_atoms=[50, 5],
                      epsilon=[0.1, 0.1], # kcal/mol
                      sigma=[2, 1], # A
                      atom_mass= [1, 1], # g/mol
                      Lx=30, # A
                      Ly=30, # A
                      Lz=30, # A
                      minimization_steps = 50,
                      maximum_steps=500,
                      desired_temperature=300,
                      thermo = 10,
                      dump = 10,  
                      tau_temp = 100, # fs
                      #tau_press= 1000, # fs
                      time_step=1, # fs
                      seed=319817,
                      data_folder = "mdcode-output/",
                      )
self.run()
# run lammps for comparison
os.system("/home/simon/Softwares/lammps-2Aug2023/src/lmp_serial -in input.lammps")

step epot maxF
0 118.148 1053.008
5 4.345 44.531
10 -0.882 0.107
15 -1.017 0.142
20 -1.129 0.468
25 -1.271 0.146
30 -1.334 0.148
35 -1.437 0.116
40 -1.487 0.140
45 -1.556 0.149
50 -1.625 0.146
 step     N  temp  epot  ekin press   vol
    0    55 299.9490346320612 -1.617139524967283 48.28085991867911 80.60144270217047 27000.0
   10    55 294.4717612331122 -0.7000400152689215 47.39921857572445 89.52882820949485 27000.0
   20    55 293.60762411258906 -0.4312643570676167 47.26012400148223 91.28510195477999 27000.0
   30    55 293.7883414307529 -0.39273696472123953 47.28921290164773 89.337801964063 27000.0
   40    55 296.5242801179417 -0.7048512025734253 47.729599291503234 82.85356552281219 27000.0
   50    55 297.4130738341511 -0.8044901547848552 47.872662678793496 78.31633193137407 27000.0
   60    55 296.86065384145024 -0.667598050142473 47.783743198467164 78.29023296882566 27000.0
   70    55 296.9425463229202 -0.629780444427926 47.79692490258931 78.30620380152855 27000.0
   80    55 

0